# Data engineering with Dask

This notebook describes the process to download and prepare United States presidential election data. You will address missing values, reformat data types, and restructure the format of a table.

***

## Load and prepare data

To download and prepare the election data, you will use ArcPy, the ArcGIS API for Python, matplotlib for visualization and a Dask dataframe. First, you will import these modules to use them. Then, you will create a variable for the United States county election data and use this variable to read the data into a Dask dataframe.

##### Import needed modules

In [172]:
import arcgis
import dask.dataframe as dd
import os
#import arcpy

##### Read data into Python

In [173]:
dask_df = dd.read_csv("countypres2016.csv", assume_missing=True)

The is usually a dtype inference failure as Dask in attempt to aid memory management takes all numeric values as 'Íntegers (int64)', this can be fixed by manually adding the dtype when reading the data or provide 'assume_missing=True' to intepret all unspecified integer columns as floats.

In [174]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/home/qwerty/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://192.168.42.173/25740/9 Dashboard: http://localhost:36203/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


The Dask Client will provide a dashboard which is useful to gain insight on the computation. The dashboard link can be seen above.

***

## Cleaning the data 

##### Exploratory Data Analysis

In [175]:
### Getting an overview of the data
dask_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016.0,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973.0,20190722.0
1,2016.0,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973.0,20190722.0
2,2016.0,Alabama,AL,Autauga,1001.0,President,Other,NaN,865.0,24973.0,20190722.0
3,2016.0,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215.0,20190722.0
4,2016.0,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215.0,20190722.0


In [176]:
# Getting overview of the the data type (dtype) 
# of all the features and get an overview of 
# features with missing values via the 'Non-Null count'
dask_df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9474 entries, 0 to 9473
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            9474 non-null   float64
 1   state           9474 non-null   object 
 2   state_po        9462 non-null   object 
 3   county          9474 non-null   object 
 4   FIPS            9462 non-null   float64
 5   office          9474 non-null   object 
 6   candidate       9474 non-null   object 
 7   party           6316 non-null   object 
 8   candidatevotes  9468 non-null   float64
 9   totalvotes      9474 non-null   float64
 10  version         9474 non-null   float64
dtypes: float64(5), object(6)
memory usage: 814.3+ KB


In an attempt to manage memory, Dask takes all the numeric values as float and non-numeric values as objects

#### Dropping redundant features

From the preview of the dataset above, it can be observed that the 'state_po' is an acronym for the 'state' feature. To make the data cleaner, we have to remove these redundant feature.

In [177]:
# dask operation
dask_df = dask_df.drop('state_po', axis=1)

In [178]:
dask_df.head()

,year,state,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016.0,Alabama,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973.0,20190722.0
1,2016.0,Alabama,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973.0,20190722.0
2,2016.0,Alabama,Autauga,1001.0,President,Other,NaN,865.0,24973.0,20190722.0
3,2016.0,Alabama,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215.0,20190722.0
4,2016.0,Alabama,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215.0,20190722.0


#### Handle missing data 

In [179]:
dask_df.isnull().sum().compute()

year                 0
state                0
county               0
FIPS                12
office               0
candidate            0
party             3158
candidatevotes       6
totalvotes           0
version              0
dtype: int64

The election data includes records that are missing data in the **,FIPS,party and candidatevotes** field. This missing data is referred to as null values. We have to ways to work with features with missing values after proper identification.
- Fill them with a value
- Remove that instance in the datasets

##### Lets investigate the features with missing values more by running queries on those features using `dask query method`

In [180]:
missing_query = dask_df.query('(FIPS == "NaN") | (candidatevotes == "NaN") ').compute()
missing_query

,year,state,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
8781,2016.0,Virginia,Bedford,51515.0,President,Hillary Clinton,democrat,NaN,0.0,20190722.0
8782,2016.0,Virginia,Bedford,51515.0,President,Donald Trump,republican,NaN,0.0,20190722.0
8783,2016.0,Virginia,Bedford,51515.0,President,Other,NaN,NaN,0.0,20190722.0
9462,2016.0,Connecticut,Statewide writein,NaN,President,Hillary Clinton,democrat,NaN,5056.0,20190722.0
9463,2016.0,Maine,Maine UOCAVA,NaN,President,Hillary Clinton,democrat,3017.0,5056.0,20190722.0
9464,2016.0,Alaska,District 99,NaN,President,Hillary Clinton,democrat,274.0,5056.0,20190722.0
9465,2016.0,Rhode Island,Federal Precinct,NaN,President,Hillary Clinton,democrat,637.0,5056.0,20190722.0
9466,2016.0,Connecticut,Statewide writein,NaN,President,Donald Trump,republican,NaN,5056.0,20190722.0
9467,2016.0,Maine,Maine UOCAVA,NaN,President,Donald Trump,republican,648.0,5056.0,20190722.0
9468,2016.0,Alaska,District 99,NaN,President,Donald Trump,republican,40.0,5056.0,20190722.0


The strategy of handling missing values that we will employ here will be replacing the missing values with a valid and representative value. 

This can be achieved with the Dask dataframe using the `fillna` method.

The 'FIPS' and 'candidatevotes' features are both numerical. In this scenario, since the data continous we could use either mean or the median would be a good representative of the central tendency of the features. In this case, we will fill the missing values with the mean of those features.

In [181]:
# Filling the missing values with the mean
dask_df["FIPS"] = dask_df["FIPS"].fillna(dask_df["FIPS"].mean().compute())
dask_df["candidatevotes"] = dask_df["candidatevotes"].fillna(dask_df["candidatevotes"].mean().compute())

In [182]:
dask_df.isnull().sum().compute()

year                 0
state                0
county               0
FIPS                 0
office               0
candidate            0
party             3158
candidatevotes       0
totalvotes           0
version              0
dtype: int64

We are left with  missing values in 'party' feature. The missing values is quite large making it critical for us to make a good choice in what to fill it with. Let's get a overview of the unique values in the feature. 

In [183]:
dask_df['party'].unique().compute()

0      democrat
1    republican
2           NaN
Name: party, dtype: object

As seen above, this depicts the voting parties in the election. To have an unbiased datasets we will fill the missing values with 'not recorded'

In [184]:
# Filling the missing values with 'not recorded'
dask_df["party"] = dask_df["party"].fillna('Others')

In [185]:
dask_df.isnull().sum().compute()

year              0
state             0
county            0
FIPS              0
office            0
candidate         0
party             0
candidatevotes    0
totalvotes        0
version           0
dtype: int64

***

## Explore and handle data types

In reviewing your data, you notice that the `FIPS` field is considered a numeric field instead of a string. As a result, leading zeroes in the FIPS values have been removed. The resulting FIPS values only have four characters instead of five. You will determine how many records are missing leading zeroes and add, or append, the missing zero.
![fix_truncated_zeroes](img/trunc_zeroes.gif "Fix Truncated Zeroes")

Also fields like `year` should be integer value rather than a float.

In [186]:
# Change the 'FIPS' field to integer firstly, to remove the decimals
dask_df['FIPS'] = dask_df['FIPS'].astype('int64')
# Then change ot to string
dask_df['FIPS'] = dask_df['FIPS'].astype('object')

# Change the 'year' field to integer
dask_df['year'] = dask_df['year'].astype('int64')

In [187]:
# Check how many records have a FIPS value with four characters
trunc_df = dask_df.loc[dask_df['FIPS'].str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / dask_df.shape[0])*100

The following cell creates a function in python that adds a leading zero to the FIPS value if it only has four characters.  

In [188]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(val)
    # Otherwise...
    else:
        # Return the value itself
        return str(val)

In [189]:
# Test the function
fix_trunc_zeros(7042)  # You should see an appended zero: "07042"

'07042'

In [190]:
# Run the function on the FIPS field using the apply and lambda method 
dask_df['FIPS'] = dask_df['FIPS'].apply(lambda x: fix_trunc_zeros(x),meta=('FIPS', 'object'))
# The metadata makes it possible for Dask not to guess the dtype 

# Print information on the operation performed, and show the first few records to confirm it worked
dask_df.head()

,year,state,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,Autauga,01001,President,Hillary Clinton,democrat,5936.0,24973.0,20190722.0
1,2016,Alabama,Autauga,01001,President,Donald Trump,republican,18172.0,24973.0,20190722.0
2,2016,Alabama,Autauga,01001,President,Other,Others,865.0,24973.0,20190722.0
3,2016,Alabama,Baldwin,01003,President,Hillary Clinton,democrat,18458.0,95215.0,20190722.0
4,2016,Alabama,Baldwin,01003,President,Donald Trump,republican,72883.0,95215.0,20190722.0


***

## Reformat the table structure

Currently, each record in the table corresponds to a candidate and their votes in a county. You need to reformat the table so that each record corresponds to each county, with fields showing the votes for different candidates in that election year. 
It is possible to do this using the [Pivot Table geoprocessing tool](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/pivot-table.htm) or Excel pivot tables, but Python may make it easier to automate and share.
The animation below illustrates the steps in restructuring the table:

The following code cell performs these steps.
![reformat_table](img/reformat_table.gif "Reformat Table")


In [216]:
c = dask_df["county"].unique().compute()
s = dask_df["state"].unique().compute()
f = dask_df["FIPS"].unique().compute()

In [192]:
p = dask_df["party"].unique().compute()

In [193]:
dd.DataFrame()

TypeError: __init__() missing 4 required positional arguments: 'dsk', 'name', 'meta', and 'divisions'

In [194]:
len(f),len(c),len(s)

(3155, 1854, 51)

In [217]:
county = dict((i,dict([("fips",None),("state",None)])) for i in list(c))

In [218]:
c

0                 Autauga
1                 Baldwin
2                 Barbour
3                    Bibb
4                  Blount
              ...        
1849               Weston
1850    Statewide writein
1851         Maine UOCAVA
1852          District 99
1853     Federal Precinct
Name: county, Length: 1854, dtype: object

In [219]:
county["Autauga"]

{'fips': None, 'state': None}

In [213]:
import pandas as pd

In [160]:
pd.DataFrame([county["Sussex"]])

,fips,state,candidate(d),votes (d),candidate(r),votes (r),candidate(O),votes (O)
0,51183,None,Hillary Clinton,2879.0,Donald Trump,2055.0,Other,110.0


In [199]:
from tqdm import tqdm

In [ ]:

i = 0
data = []

for row in tqdm(range(len(dask_df))):
    
    df = dask_df.compute()
    
    c = df.loc[row,"county"]
    s = df.loc[row,"state"]
    f = df.loc[row,"FIPS"]
    
    can_nm = df.loc[row, "candidate"]
    party =  df.loc[row, "party"]
    votes =  df.loc[row, "candidatevotes"]
    
    if f != county[c]["fips"]:
        county[c]["fips"] = f
    
    county[c][f"candidate({party.strip()[0]})"] = can_nm
    county[c][f"votes ({party.strip()[0]})"] = votes
    county[c]['county'] = c
    
    

 45%|████▍     | 4257/9474 [06:42<08:09, 10.66it/s]distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
 45%|████▌     | 4267/9474 [06:43<08:22, 10.36it/s]distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
 45%|████▌     | 4269/9474 [06:43<08:28, 10.24it/s]distributed.worker - WARNING - Memory use is high but worker has no

In [130]:
if "Autauga" in county.keys():
    print("f")

f


In [72]:
v = dask_df["county"].compute()

In [126]:
dask_df.compute().iloc[1,3]

'01001'

In [88]:
b = c.astype("object")

In [45]:
# Let's create a new Dask dataframe
df_out = dask_df[['FIPS','year','county','state','office', 'candidate', 'candidatevotes']]

In [46]:
# Democrates features initialization
df_out['candidate_dem'] = 0
df_out['voters_dem'] = 0

#Republician features initialization
df_out['candidate_rep'] = 0
df_out['voters_rep'] = 0

In [47]:
df_out.head()

,FIPS,year,county,state,office,candidate,candidatevotes,candidate_dem,voters_dem,candidate_rep,voters_rep
0,01001,2016,Autauga,Alabama,President,Hillary Clinton,5936.0,0,0,0,0
1,01001,2016,Autauga,Alabama,President,Donald Trump,18172.0,0,0,0,0
2,01001,2016,Autauga,Alabama,President,Other,865.0,0,0,0,0
3,01003,2016,Baldwin,Alabama,President,Hillary Clinton,18458.0,0,0,0,0
4,01003,2016,Baldwin,Alabama,President,Donald Trump,72883.0,0,0,0,0


In [48]:
# For quick trials
dff_out = df_out.copy()

In [49]:
dff_out.head()

,FIPS,year,county,state,office,candidate,candidatevotes,candidate_dem,voters_dem,candidate_rep,voters_rep
0,01001,2016,Autauga,Alabama,President,Hillary Clinton,5936.0,0,0,0,0
1,01001,2016,Autauga,Alabama,President,Donald Trump,18172.0,0,0,0,0
2,01001,2016,Autauga,Alabama,President,Other,865.0,0,0,0,0
3,01003,2016,Baldwin,Alabama,President,Hillary Clinton,18458.0,0,0,0,0
4,01003,2016,Baldwin,Alabama,President,Donald Trump,72883.0,0,0,0,0


In [50]:
dff_out.index.compute()

RangeIndex(start=0, stop=9474, step=1)

In [52]:
for ind in dff_out.index.compute():
    if (dff_out['candidate'] == 'Hillary Clinton').all():
        dff_out['voters_dem'] = dff_out['candidatevotes']
    
    elif (dff_out['candidate'] == 'Donald Trump').all():
        dff_out['voters_rep'] = dff_out['candidatevotes']

In [54]:
# Set an index using mulitple fields, which "locks" these fields before the table pivots
# Use the built-in groupby function for the FIPS and year fields, which you use to group the data by candidate
# Use unstack to perform the table pivot, which will rotate the table and turn rows into columns
df_out = dask_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            dask_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

NotImplementedError: Dask dataframe does not yet support multi-indexes.
You tried to index with this index: ['FIPS', 'year', 'county', 'state', 'state_po', 'office', Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: add, 33 tasks]
Indexes must be single columns only.

In [53]:
dask_df.compute().shape

(9474, 10)

In [64]:
fips_value = dask_df['FIPS'].compute().unique()
fips_list = []
for fip in fips_value:
    fips_list.append(fip)

In [66]:
for row in fips_list:
    if (dask_df['candidate'] == 'Hillary Clinton').all():
        dask_df['voters_dem'] = dask_df['candidatevotes']
    
    elif (dask_df['candidate'] == 'Donald Trump').all():
        dask_df['voters_rep'] = dask_df['candidatevotes']

In [ ]:
if (dask_df['candidate'] == 'Hillary Clinton').all():
    dask_df['voters_dem'] = dask_df['candidatevotes']
elif (dask_df['candidate'] == 'Donald Trump').all()
    dask_df['voters_rep'] = dask_df['totalvotes'] - dask_df['candidatevotes']

In [27]:
a = dask_df[dask_df['county'] == 'Autauga']

In [29]:
a.head(10)

,year,state,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,Autauga,01001,President,Hillary Clinton,democrat,5936.0,24973.0,20190722.0
1,2016,Alabama,Autauga,01001,President,Donald Trump,republican,18172.0,24973.0,20190722.0
2,2016,Alabama,Autauga,01001,President,Other,not recorded,865.0,24973.0,20190722.0
3,2016,Alabama,Autauga,01003,President,Hillary Clinton,democrat,18458.0,95215.0,20190722.0
4,2016,Alabama,Autauga,01003,President,Donald Trump,republican,72883.0,95215.0,20190722.0
5,2016,Alabama,Autauga,01003,President,Other,not recorded,3874.0,95215.0,20190722.0
6,2016,Alabama,Autauga,01005,President,Hillary Clinton,democrat,4871.0,10469.0,20190722.0
7,2016,Alabama,Autauga,01005,President,Donald Trump,republican,5454.0,10469.0,20190722.0
8,2016,Alabama,Autauga,01005,President,Other,not recorded,144.0,10469.0,20190722.0
9,2016,Alabama,Autauga,01007,President,Hillary Clinton,democrat,1874.0,8819.0,20190722.0


In [56]:
# Set an index using mulitple fields, which "locks" these fields before the table pivots
# Use the built-in groupby function for the FIPS and year fields, which you use to group the data by candidate
# Use unstack to perform the table pivot, which will rotate the table and turn rows into columns
df_out = dask_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state',  
                            'office', 
                            dask_df.groupby(by ='FIPS','year').cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

SyntaxError: positional argument follows keyword argument (<ipython-input-56-d79f13dc1724>, line 9)

In [19]:
dask_df.head(1)

,year,state,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2016,Alabama,Autauga,01001,President,Hillary Clinton,democrat,5936.0,24973.0,20190722.0


In [37]:
df_out.head()

,FIPS,year,county,state,office,candidate,totalvotes
0,01001,2016,Autauga,Alabama,President,Hillary Clinton,24973.0
1,01001,2016,Autauga,Alabama,President,Donald Trump,24973.0
2,01001,2016,Autauga,Alabama,President,Other,24973.0
3,01003,2016,Baldwin,Alabama,President,Hillary Clinton,95215.0
4,01003,2016,Baldwin,Alabama,President,Donald Trump,95215.0


In [44]:
df_out.groupby(by='FIPS').agg('sum').compute()

,year,county,state,office,candidate,totalvotes
FIPS,,,,,,
01001,6048,AutaugaAutaugaAutauga,AlabamaAlabamaAlabama,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,74919.0
01003,6048,BaldwinBaldwinBaldwin,AlabamaAlabamaAlabama,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,285645.0
01005,6048,BarbourBarbourBarbour,AlabamaAlabamaAlabama,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,31407.0
01007,6048,BibbBibbBibb,AlabamaAlabamaAlabama,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,26457.0
01009,6048,BlountBlountBlount,AlabamaAlabamaAlabama,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,76764.0
...,...,...,...,...,...,...
56037,6048,SweetwaterSweetwaterSweetwater,WyomingWyomingWyoming,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,51390.0
56039,6048,TetonTetonTeton,WyomingWyomingWyoming,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,37881.0
56041,6048,UintaUintaUinta,WyomingWyomingWyoming,PresidentPresidentPresident,Hillary ClintonDonald TrumpOther,25410.0


In [ ]:
df.groupby('name').agg({'x': ['mean', 'std'], 'y': ['mean', 'count']}).compute().head()

In [33]:
df_out.groupby(by='candidate').count().compute()

,FIPS,year,county,state,office,totalvotes
candidate,,,,,,
Donald Trump,3158,3158,3158,3158,3158,3158
Hillary Clinton,3158,3158,3158,3158,3158,3158
Other,3158,3158,3158,3158,3158,3158


In [35]:
dask_df.groupby(by='county').count().compute()

,year,state,FIPS,office,candidate,party,candidatevotes,totalvotes,version
county,,,,,,,,,
Abbeville,3,3,3,3,3,2,3,3,3
Acadia,3,3,3,3,3,2,3,3,3
Accomack,3,3,3,3,3,2,3,3,3
Ada,3,3,3,3,3,2,3,3,3
Adair,12,12,12,12,12,8,12,12,12
...,...,...,...,...,...,...,...,...,...
Yuba,3,3,3,3,3,2,3,3,3
Yuma,6,6,6,6,6,4,6,6,6
Zapata,3,3,3,3,3,2,3,3,3


In [25]:
df_out.head()

,FIPS,year,county,state,office
0,01001,2016,Autauga,Alabama,President
1,01001,2016,Autauga,Alabama,President
2,01001,2016,Autauga,Alabama,President
3,01003,2016,Baldwin,Alabama,President
4,01003,2016,Baldwin,Alabama,President


In [32]:
dask_df['candidate'].unique().compute()

0    Hillary Clinton
1       Donald Trump
2              Other
Name: candidate, dtype: object

In [27]:
dask_df['candidate_dem'] = 'Hillary Clinton'

In [37]:
if (dask_df['candidate'] == 'Hillary Clinton').all():
    dask_df['voters_dem'] = dask_df['candidatevotes']
elif (dask_df['candidate'] == 'Donald Trump').all()
    dask_df['voters_rep'] = dask_df['totalvotes'] - dask_df['candidatevotes']

SyntaxError: invalid syntax (<ipython-input-37-6985bebceba2>, line 3)

In [35]:
dask_df.head()

,year,state,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,candidate_dem,voters_dem
0,2016,Alabama,Autauga,01001,President,Hillary Clinton,democrat,5936.0,24973.0,20190722.0,Hillary Clinton,5936.0
1,2016,Alabama,Autauga,01001,President,Donald Trump,republican,18172.0,24973.0,20190722.0,Hillary Clinton,18172.0
2,2016,Alabama,Autauga,01001,President,Other,not recorded,865.0,24973.0,20190722.0,Hillary Clinton,865.0
3,2016,Alabama,Baldwin,01003,President,Hillary Clinton,democrat,18458.0,95215.0,20190722.0,Hillary Clinton,18458.0
4,2016,Alabama,Baldwin,01003,President,Donald Trump,republican,72883.0,95215.0,20190722.0,Hillary Clinton,72883.0


***

## Reformat the table structure

In [27]:
# Set an index using mulitple fields, which "locks" these fields before the table pivots
# Use the built-in groupby function for the FIPS and year fields, which you use to group the data by candidate
# Use unstack to perform the table pivot, which will rotate the table and turn rows into columns
df_out = dask_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            dask_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

NotImplementedError: Dask dataframe does not yet support multi-indexes.
You tried to index with this index: ['FIPS', 'year', 'county', 'state', 'state_po', 'office', Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: add, 35 tasks]
Indexes must be single columns only.

## Calculate additional columns: Feature Engineering